# Benchmarking Gurobi


In [ ]:
%matplotlib notebook
%load_ext rpy2.ipython
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from pyvis.network import Network
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
readRDS = robjects.r['readRDS']


class SolverLogs(object):
    def __init__(self, filenames):
        self.__data__ = {x: getattr(self, f"__get_{x}__")(y) for x, y in filenames.items()}

    def __getitem__(self, key):
        return self.__data__[key]
    
    def get_data(self):
        return self.__data__
        
    def __get_cbc__(self, path_to_log):
        obj_val = 0
        with open(path_to_log) as fh:
            for line in fh:
                if "Objective value:" in line:
                    obj_val = float(line.split("Objective value:")[-1].strip("\n "))
        return {"objective_value": obj_val, "solution_count": 1}
    
    def __get_cplex__(self, path_to_log):
        obj_val = 0
        sol_count = 0
        with open(path_to_log) as fh:
            for line in fh:
                if "Objective = " in line:
                    obj_val = float(line.split("Objective = ")[-1].strip("\n "))
                if "Solution pool:" in line:
                    sol_count = int(line.split(" ")[2])
        return {"objective_value": obj_val, "solution_count": sol_count}

    def __get_gurobi__(self, path_to_log):
        obj_val = 0
        sol_count = 0
        with open(path_to_log) as fh:
            for line in fh:
                if "Best objective" in line:
                    obj_val = float(line.split(",")[0].split("Best objective")[-1].strip("\n "))
                if "Solution count" in line:
                    sol_count = int(line.split(":")[0].split(" ")[-1])
        return {"objective_value": obj_val, "solution_count": sol_count}

# Example 

In [ ]:
fname = "Output/Erdos/E300_N100_I10_M10_S1_P2_2/graph.dot"
g = Network(500, 1000, notebook=True)
g.from_DOT(fname)
g.show(fname.replace(".dot", ".html"))

In [ ]:
solvers = ("cbc", "cplex", "gurobi")

logs = {x: fname.replace("graph.dot", f"{x}/log.txt") for x in solvers}
logs = SolverLogs(logs).get_data()

benchmarks = {x: pd.read_csv(fname.replace("graph.dot", f"{x}/benchmark.tsv"), "\t") 
              for x in solvers}

fig, axs = plt.subplots(1, 4, figsize=(12, 4))
fig.subplots_adjust(left=0.1, right=0.9, wspace=0.4)
axs[0].bar(solvers, [benchmarks[x]["s"].mean()/60 for x in solvers])
axs[0].set_ylabel("Execution time [min]")
axs[1].bar(solvers, [benchmarks[x]["max_rss"].mean() for x in solvers])
axs[1].set_ylabel("Memory usage [MB]")
axs[2].bar(solvers, [logs[x]["objective_value"] for x in solvers])
axs[2].set_ylabel("Objective function value")
axs[3].bar(solvers, [logs[x]["solution_count"] for x in solvers])
axs[3].set_ylabel("Numner of solutions")

Solution given by CARNIVAL:

In [ ]:
solver = "cplex"
fname = fname.replace("graph.dot", f"{solver}/network_solution.dot")
g = Network(500, 1000, notebook=True)
g.from_DOT(fname)
g.show(fname.replace(".dot", ".html"))

# Results